In [0]:
#load the data from the google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# If this code runs and says "Found GPU ..." etc then congrats, you've turned the computation machine to full volume

import tensorflow as tf # Importing our first module (as below) but we need it 
                        # earlier to check whether we have the GPU running in the correct place!
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#Import the necessary libraries

import os
import numpy as np
from tqdm import tqdm #just to show the progress bar
import sys
import random
from skimage.io import imread,imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from numpy import asarray
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from skimage.transform import rotate
from skimage import color
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


Using TensorFlow backend.


In [0]:
#Different metrics that can be used for segmentation

def dice(y_true, y_pred):
        eps = 0.00001 
        y_true_f = tf.reshape(y_true,[tf.shape(y_true)[0],tf.shape(y_true)[1]*tf.shape(y_true)[2]])
        y_pred_f = tf.reshape(y_pred,[tf.shape(y_pred)[0],tf.shape(y_pred)[1]*tf.shape(y_pred)[2]])                                                                                                
        intersection = eps + 2*tf.reduce_sum(y_true_f*y_pred_f, axis=1)                                                    
        # eps added in denomintor, to take care for DivisionByZero error.
        union = eps + tf.reduce_sum(y_true_f*y_true_f, axis=1) + tf.reduce_sum(y_pred_f*y_pred_f, axis=1)                                    
        IOU = tf.math.divide(intersection,union)                                                                               
        return (tf.reduce_mean(IOU))
         
def dice_loss(y_true, y_pred):
        return -dice(y_true, y_pred)

def dice_metric(y_true, y_pred):
        y_true_f = K.cast(K.greater(y_true, 0.5), 'float32')
        y_pred_f = K.cast(K.greater(y_pred, 0.5), 'float32')
        
        return dice(y_true_f, y_pred_f)

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

In [0]:
#install the albumentations library
pip install -U git+https://github.com/albu/albumentations

In [0]:
#Import the augmentation tools
from albumentations import (
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,    
    RandomGamma    
)

In [0]:
#procedure for visualising different augmentations applied to an image
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image, cmap='gray', vmin=0, vmax=255)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image, cmap='gray', vmin=0, vmax=255)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)

In [0]:
image=imread("/content/drive/My Drive/3YP_Machine_Learning/PhC-C2DH-U373/01/t001.tif")
mask=imread("/content/drive/My Drive/3YP_Machine_Learning/PhC-C2DH-U373/01_ST/SEG/man_seg001.tif")

In [0]:
#The procedure that applies augmentations to an image and mask where X is a training dataset and Y is the ground truth dataset
def augmentation(image, mask, X, Y):

 #first degree of augmentations including the rotations...
  image_medium1 = image[::-1, :] 
  mask_medium1 = mask[::-1, :] 
  #append them to the data set
  X.append(image_medium1)
  Y.append(mask_medium1)
  image_medium10 = image[:, ::-1]
  mask_medium10 = mask[:, ::-1]
  #append them to the data set
  image_medium10=img_to_array(image_medium10)
  mask_medium10=img_to_array(mask_medium10)
  X.append(image_medium10)
  Y.append(mask_medium10)    
  image_medium2 = rotate(image, 10)
  mask_medium2 = rotate(mask, 10)
  #append them to the data set
  image_medium2=img_to_array(image_medium2)
  mask_medium2=img_to_array(mask_medium2)
  X.append(image_medium2)
  Y.append(mask_medium2)
  image_medium7 = rotate(image, -10)
  mask_medium7 = rotate(mask, -10)
  #append them to the data set
  image_medium7=img_to_array(image_medium10)
  mask_medium7=img_to_array(mask_medium10)
  X.append(image_medium7)
  Y.append(mask_medium7)
  image_medium8 = rotate(image, 20)
  mask_medium8 = rotate(mask, 20)
  #append them to the data set
  image_medium8=img_to_array(image_medium8)
  mask_medium8=img_to_array(mask_medium8)
  X.append(image_medium8)
  Y.append(mask_medium8)
  image_medium9 = rotate(image, -20)
  mask_medium9 = rotate(mask, -20)
  #append them to the data set
  image_medium9=img_to_array(image_medium9)
  mask_medium9=img_to_array(mask_medium9)
  X.append(image_medium9)
  Y.append(mask_medium9)
  #second level of the augmentations including elastic deformations
  aug3 = ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
  augmented3 = aug3(image=image, mask=mask)
  image_medium3 = augmented3['image']
  mask_medium3 = augmented3['mask']
  #append them to the dataset
  image_medium3=img_to_array(image_medium3)
  mask_medium3=img_to_array(mask_medium3)
  X.append(image_medium3)
  Y.append(mask_medium3)
  aug4 = GridDistortion(p=1.5)
  augmented4 = aug4(image=image, mask=mask)
  image_medium4 = augmented4['image']
  mask_medium4 = augmented4['mask']
  #append them to the dataset
  image_medium4=img_to_array(image_medium4)
  mask_medium4=img_to_array(mask_medium4)
  X.append(image_medium4)
  Y.append(mask_medium4)
  #aug5 = OpticalDistortion(p=1, distort_limit=1, shift_limit=0.5)
  #augmented5 = aug5(image=image, mask=mask)
  #image_medium5 = augmented5['image']
  #mask_medium5 = augmented5['mask']
  #append them to the dataset
  

In [0]:
#This part of the code performs data loading and prepares the data for the neural netork

from tensorflow.keras.optimizers import Adam


np.random.seed(123)

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

TRAIN_PATH = "/content/drive/My Drive/3YP_Machine_Learning/PhC-C2DH-U373/01"
MASKS_PATH = "/content/drive/My Drive/3YP_Machine_Learning/PhC-C2DH-U373/01_ST/SEG"
TEST_DATA= "/content/drive/My Drive/3YP_Machine_Learning/PhC-C2DH-U373_2/01"

trin_ids= os.listdir(TRAIN_PATH)
test_ids= os.listdir(TEST_DATA)
mask_ids= os.listdir(MASKS_PATH)

X_train=list()
Y_train=list()

#loading in the training data

for n in range(91):
  image=imread(TRAIN_PATH +'/'+ train_ids[n])
  image=resize(image, (IMG_HEIGHT,IMG_WIDTH,1))
  image=img_to_array(image)
  X_train.append(image)
  if n<10:
    s = '/man_seg00'+ str(n) +'.tif'
  else:
    s='/man_seg0' + str(n) + '.tif'
  mask=65535*imread(MASKS_PATH +'/'+ mask_ids[n])
  mask= resize(image, (IMG_HEIGHT,IMG_WIDTH,1))
  mask=img_to_array(mask)
  Y_train.append(mask)
  augmentation(image, mask, X_train, Y_train)

X_Train = np.array(X_train)
Y_Train = np.array(Y_train)
 #loading in the test data
X_test=list()
for i in range(10):
  if i<10:
    s = '/t00'+ str(i) +'.tif'
  else:
    s='/t0' + str(i) + '.tif'
  image_test=imread(TEST_DATA + s)
  image_test=resize(image, (IMG_HEIGHT,IMG_WIDTH,1))
  image_test=img_to_array(image_test)
  X_test.append(image_test)

X_Test=np.array(X_test)

In [0]:
#Sanity check is the data in the right format, if it is (no_of_images, size_x, size_y, number of channels).....then yes!
print(X_Train.shape, Y_Train.shape, X_Test.shape)

(819, 128, 128, 1) (819, 128, 128, 1) (10, 128, 128, 1)


In [0]:
#Build the model
inputs = tf.keras.layers.Input((128, 128,1))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['jacard_coef'])
model.summary()

NameError: ignored

In [0]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('hosico1.h5', verbose=1)
#callbacks1=[
           #tf.keras.callbacks.EarlyStopping(patience=2,monitor='val_loss'),
           #tf.keras.callbacks.TensorBoard(log_dir='logs')
#]
results = model.fit(X_Train, Y_Train,batch_size=32, epochs=100,callbacks=checkpointer, shuffle=True) #choose suffle=true to shuffle the training images order

Epoch 1/100
203/205 [============================>.] - ETA: 0s - loss: 0.6922 - accuracy: 0.0331
Epoch 00001: saving model to hosico1.h5
205/205 [==============================] - 3s 14ms/step - loss: 0.6922 - accuracy: 0.0332
Epoch 2/100
202/205 [============================>.] - ETA: 0s - loss: 0.6900 - accuracy: 0.0330
Epoch 00002: saving model to hosico1.h5
205/205 [==============================] - 3s 14ms/step - loss: 0.6900 - accuracy: 0.0333
Epoch 3/100
203/205 [============================>.] - ETA: 0s - loss: 0.6859 - accuracy: 0.0331
Epoch 00003: saving model to hosico1.h5
205/205 [==============================] - 3s 13ms/step - loss: 0.6859 - accuracy: 0.0333
Epoch 4/100
204/205 [============================>.] - ETA: 0s - loss: 0.6811 - accuracy: 0.0334
Epoch 00004: saving model to hosico1.h5
205/205 [==============================] - 3s 14ms/step - loss: 0.6811 - accuracy: 0.0333
Epoch 5/100
202/205 [============================>.] - ETA: 0s - loss: 0.6758 - accuracy: 0.